### Load Pickle file

In [ ]:
import pickle
from pathlib import Path

root_module = Path.cwd()
ext = ".pkl"
names = ["deep_sea_ensemble", "nroom_ensemble", "sparse_pendulum_ensemble"]
load_dir = root_module.parent.joinpath("data/")
data = {}
for name in names:
    file_dir = load_dir.joinpath(name + ext)
    data[name] = pickle.load(open(file_dir, "rb"))

### Get metrics from raw data

In [ ]:
import numpy as np
from itertools import product
from scipy.stats import sem

# Deep Sea
data["deep_sea_ensemble"]["mean_regret"] = {}
data["deep_sea_ensemble"]["ci_regret"] = {}
for key in data["deep_sea_ensemble"]["total_regret"].keys():
    data["deep_sea_ensemble"]["mean_regret"][key] = np.mean(data["deep_sea_ensemble"]["total_regret"][key], axis=-1)
    data["deep_sea_ensemble"]["ci_regret"][key] = sem(data["deep_sea_ensemble"]["total_regret"][key], axis=-1)

# Nroom
data["nroom_ensemble"]["mean_regret"] = {}
data["nroom_ensemble"]["ci_regret"] = {}
for key in data["nroom_ensemble"]["total_regret"].keys():
    data["nroom_ensemble"]["mean_regret"][key] = np.mean(data["nroom_ensemble"]["total_regret"][key], axis=-1)
    data["nroom_ensemble"]["ci_regret"][key] = sem(data["nroom_ensemble"]["total_regret"][key], axis=-1)

# Sparse pendulum

# Get available methods and noise levels to plot
keys_arr = np.array(list((data["sparse_pendulum_ensemble"]["returns"].keys())), dtype=object)
pendulum_agent_types = np.array(list(set(keys_arr[:, 0])), dtype=tuple)
pendulum_agent_types = pendulum_agent_types[pendulum_agent_types.argsort()]
# rearrange to desire order 
order = [1, 3, 0, 2]
pendulum_agent_types = pendulum_agent_types[order].tolist()
noise_std = np.sort(list(set(keys_arr[:, 1])))
ensemble_sizes = np.sort(list(set(keys_arr[:, 2].astype(int))))

data["sparse_pendulum_ensemble"]["mean_final_returns"] = {}
data["sparse_pendulum_ensemble"]["ci_final_returns"] = {}
for type, noise, size in product(pendulum_agent_types, noise_std, ensemble_sizes):
    idx = (type, noise, str(size))
    final_returns = data["sparse_pendulum_ensemble"]["returns"][idx][-1]
    data["sparse_pendulum_ensemble"]["mean_final_returns"][idx] = np.mean(final_returns)
    data["sparse_pendulum_ensemble"]["ci_final_returns"][idx] = sem(final_returns)

### Plots

In [ ]:
%matplotlib widget
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import product
from ube_mbrl.utils.plot import PARAMS

plt.rcParams.update(PARAMS)

# Get unique agent types and deep sea sizes in the data
keys_arr = np.array(list((data["deep_sea_ensemble"]["total_regret"].keys())), dtype=object)

# convert agent types to array so we can sort it and order it how we like
agent_types = np.array(list(set(keys_arr[:, 0])), dtype=tuple)
agent_types = agent_types[agent_types[:,1].argsort()]
# rearrange to desire order 
order = [1, 2, 0]
agent_types = agent_types[order].tolist()
# convert back to tuple so we use it as index
agent_types = [tuple(agent_type) for agent_type in agent_types]

tabular_ensemble_sizes = np.sort(list(set(keys_arr[:, 1].astype(int))))
deep_sea_sizes = np.sort(list(set(keys_arr[:, -1])))

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(7.5, 1.5),gridspec_kw={'wspace':0.25,'hspace':0.1})

# Tabular plots
tabular_colors = {}
cmap = plt.get_cmap("tab10")
for i, agent_type in enumerate(agent_types):
    tabular_colors[agent_type] = cmap(i)

ax_deep_sea = axes[0]
ax_nroom = axes[1]
wd = 0.2
pos = [-wd, 0, wd]
order = [0, 1, 2]
xticks = list(range(1, len(tabular_ensemble_sizes) + 1))
xticks_labels = set()
for i, agent in enumerate(agent_types):
    for ensemble_size, xtick in zip(tabular_ensemble_sizes, xticks):
        xticks_labels.add(str(ensemble_size))
        deep_sea_size_to_plot = '20'
        idx = (agent, str(ensemble_size), deep_sea_size_to_plot)
        ax_deep_sea.bar(
            xtick + pos[order[i]],
            data["deep_sea_ensemble"]["mean_regret"][idx][-1], 
            yerr=data["deep_sea_ensemble"]["ci_regret"][idx][-1], 
            width=wd, 
            color=tabular_colors[agent]
        )
        idx = (agent, str(ensemble_size))
        ax_nroom.bar(
            xtick + pos[order[i]],
            data["nroom_ensemble"]["mean_regret"][idx][-1], 
            yerr=data["nroom_ensemble"]["ci_regret"][idx][-1], 
            width=wd, 
            color=tabular_colors[agent]
        )

# Labels and titles
labels = np.sort(np.array(list(xticks_labels), dtype=int))
ax_deep_sea.set_title("DeepSea")
ax_deep_sea.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax_deep_sea.set_xticks(ticks=xticks, labels=labels)
ax_deep_sea.set_ylabel("Total regret")
ax_nroom.set_title("7-room")
ax_nroom.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax_nroom.set_xticks(ticks=xticks, labels=labels)
ax_nroom.set_ylabel("Total regret")

# Pendulum plot
ax_pendulum = axes[2]
colors = {}
for i, type in enumerate(pendulum_agent_types):
    colors[type] = cmap(i)

wd = 0.2
min_level = -70.0
pos = [-3 * wd/2, -wd/2, wd/2, 3*wd/2]
xticks = list(range(1, len(ensemble_sizes) + 1))
xticks_labels = set()
for (i, type), (ensemble_size, xtick) in product(enumerate(pendulum_agent_types), zip(ensemble_sizes, xticks)):
    noise_std = '0.0'
    idx = (type, noise_std,  str(ensemble_size))
    xticks_labels.add(str(ensemble_size))
    ax_pendulum.bar(
        xtick + pos[i],
        data["sparse_pendulum_ensemble"]["mean_final_returns"][idx] - min_level,
        yerr=data["sparse_pendulum_ensemble"]["ci_final_returns"][idx],
        width=wd,
        color=colors[type],
        bottom=min_level
    )

patches = []
for type in pendulum_agent_types:
    if type == "exact_ube_3":
        label = r"\texttt{exact-ube}" + " (ours)"
    elif type == "ensemble":
        label = r"\texttt{ensemble-var}"
    elif type == "none":
        label = r"\texttt{ensemble-mean}"
    else: label = fr"\texttt{{{type}}}"
    patches.append(mpatches.Patch(color=colors[type], label=label))


labels = np.sort(np.array(list(xticks_labels), dtype=int))
ax_pendulum.legend(handles=patches, loc = 'lower center', ncol=4, bbox_to_anchor=(-0.8, -0.6), frameon=False)
ax_pendulum.set_ylabel("Return")
ax_pendulum.set_xticks(ticks=xticks, labels=labels)
ax_pendulum.set_title("Pendulum")
ax_pendulum.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
ax_pendulum.set_ylim(bottom=min_level, top=260)

for ax in axes:
    ax.set_xlabel("Ensemble size")

### Save figures

In [ ]:
# Save figures
import os
import pickle
from pathlib import Path
root_module = Path.cwd()
fig_dir = root_module.parent.joinpath(f"figures/ensemble_ablation.pdf")
fig.savefig(fig_dir, bbox_inches="tight", transparent=False)

# License

>Copyright (c) 2023 Robert Bosch GmbH
>
>This program is free software: you can redistribute it and/or modify <br>
>it under the terms of the GNU Affero General Public License as published<br>
>by the Free Software Foundation, either version 3 of the License, or<br>
>(at your option) any later version.<br>
>
>This program is distributed in the hope that it will be useful,<br>
>but WITHOUT ANY WARRANTY; without even the implied warranty of<br>
>MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the<br>
>GNU Affero General Public License for more details.<br>
>
>You should have received a copy of the GNU Affero General Public License<br>
>along with this program.  If not, see <https://www.gnu.org/licenses/>.